In [7]:
#Importing 
import os
import pandas as pd
import re
from datetime import timedelta
from datetime import datetime ,timedelta
import pickle

#Five recent logs
data = pd.read_csv('logfiles.csv', index_col = None)
#DroppingNA
data = data[~data['logfile'].isna()]
data = data[~data['exitcode'].isna()]
#Formatting
data['logfile']= data['logfile'].map(str)
data['start_time'] = pd.to_datetime(data['start_time'])
data['logfile'] = data['logfile'].apply(lambda x: int(re.split('-|_', x)[-1].replace('.log','')))
#Checking errors
def result(row):
    if row['exitcode'] == 0:
        return row['start_time'] + timedelta(hours=7) 
    else:
        return 'failed, exit code = {0}'.format(row['exitcode'])
#Results of recent logs
def recentlogs(subset):
    #GettingScriptName
    Script = subset['jobname']
    Script = Script.iloc[0,]
    #SortingFiveRecentLogs
    Recent_log = subset.sort_values(by=['start_time'], ascending = False).head(5)
    Recent_log['exitcode'] = Recent_log['exitcode'].astype('int')
    values = Recent_log.apply(result, axis = 1)
    values = pd.DataFrame(values)
    #Formatting
    values = values.iloc[::-1]
    values.columns = [Script]
    report = values.transpose()
    report.columns = ['Recent 5th Run', 'Recent 4th Run','Recent 3rd Run','Recent 2nd Run','Last Run']
    return report
Recent_log = pd.DataFrame(columns = ['Recent 5th Run', 'Recent 4th Run','Recent 3rd Run','Recent 2nd Run','Last Run'])
#ConcatingDifferentJobnames
for i in data['jobname'].unique():
    Recent_log = pd.concat([Recent_log, recentlogs(data[data['jobname']==i])])
#FolderandFileTable
folders = [f.name for f in os.scandir(r'\\VNATSHFS.intel.com\VNATAnalysis$\MAOATM\VN\Applications\ICM_PCS')]
filenames = []
for i in folders:
    subfile = [f.name for f in os.scandir(os.path.join(r'\\VNATSHFS.intel.com\VNATAnalysis$\MAOATM\VN\Applications\ICM_PCS',i))]
    filenames.append(subfile[0])
filenames = [re.sub('_\d+\s*.log','',x).replace('.log','') for x in filenames]
#Reindexing
dictionary = dict(zip(filenames, folders))
Recent_log.rename(index = dictionary)
s = Recent_log.index.to_series()
Recent_log.index = s.map(dictionary).fillna(s)
Recent_log

,Recent 5th Run,Recent 4th Run,Recent 3rd Run,Recent 2nd Run,Last Run
PPV_Cfuse_ISL_hold_V8,2021-04-14 03:04:45,2021-04-14 04:35:10,2021-04-14 06:01:48,2021-04-14 07:30:33,2021-04-14 09:03:12
STHI_CGM_Potential_wrapper,"failed, exit code = 1","failed, exit code = 1","failed, exit code = 1","failed, exit code = 1","failed, exit code = 1"


In [9]:
#----------------------------
#GettingErrorLogs

import os
import re
import pandas as pd
from datetime import datetime ,timedelta
import pickle

# function to get ID from files
def get_ID(foldername,subfile):

    IDString = subfile.split("_")
    if len(IDString) == 1:
        IDString = subfile.split("-")

    ID = int(IDString[-1].replace(".log",""))
    
    return ID 

# function to extract data from files
def read_extract(newpath, subfile,ID):
    error = False
    path = os.path.join(newpath, subfile)
    f = open(path, "r")
    for line in f:
        # find date
        if re.findall("Local DateTime:",line):
            time = line

        # find if the file has any error
        if re.findall("error", line.lower()):
            error = True
    
    # format time
    time = time.replace("Local DateTime: ","").replace("\n","")
    time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    return [ID,time,error]


# create a file that can store read files 
try:
    # open the file if it already existed 
    a_file = open("runnedID.pkl", "rb")
    recently_checked = pickle.load(a_file)
except FileNotFoundError:
    # create open if none exised 
    a_file = open("runnedID.pkl","w+")
    recently_checked = {}


# the directory contains the log files
directory = r"\\VNATSHFS.intel.com\VNATAnalysis$\MAOATM\VN\Applications\ICM_PCS"

# A empty dataframe for final return
report_frame = pd.DataFrame(columns=["Last log with error","Last log time"])

# -------------------------------------------------
# code start here

# enter folder 
for foldername in os.listdir(directory):

    # newpath to that after enter one folder
    newpath = os.path.join(directory, foldername)

    # check if it is a directory or not
    if os.path.isdir(newpath):
        

        # create a temp dataframe to contain a folder data
        df = pd.DataFrame(columns=["ID","Date", "Error"])


        
        # extract checked ID from previous runs
        # minID = 0
        try:
            minID = recently_checked[foldername]
        except KeyError:
            minID = 0



        # enter the log files
        for subfile in os.listdir(newpath):
            
            # check if the file end with log or not
            if subfile.endswith(".log") and subfile.count(".") == 1:

                ID = get_ID(foldername,subfile)
            
            # examine if the log file is checked from the previous run
            if ID <= minID: 
                continue
            else:
                # if not extract info from the file
                minID = ID

                # extract and append the data to the temp dataframe
                df.loc[df.shape[0]] = read_extract(newpath,subfile,ID)


        # If all file is examined, append None
        if df.empty:
            report_frame.loc[foldername] = [None,None]
            continue 

        # sort data by date
        df = df.sort_values(by=["Date"],ascending = False)
        print(df)


        # select any log that have error 
        Errors = df[df["Error"] == True]

        # if the Error subset is empty return None else get the most recent date that got an error
        if not Errors.empty:
            outputError  = Errors.iloc[0,:]["Date"] 
        else:
            outputError = None

        # Get the most recent runs
        Recent = []
        if df.iloc[0,:]["Date"]:
            Recent.append(df.iloc[0,:]["Date"])
        
        
        # save the checked ID from above
        recently_checked[foldername] = minID

        # append error and recent data to the final dataframe
        report_frame.loc[foldername] = [outputError] + Recent
        print(report_frame)


# same the new ID  
a_file = open("runnedID.pkl", "wb")
pickle.dump(recently_checked, a_file)
a_file.close()


TypeError: '<=' not supported between instances of 'int' and 'dict'

In [10]:
#-----------------------------
 #Concatenate 
Full_Report = pd.concat([report_frame, Recent_log], axis = 1)
Full_Report

,Last log with error,Last log time,Recent 5th Run,Recent 4th Run,Recent 3rd Run,Recent 2nd Run,Last Run
PPV_Cfuse_ISL_hold_V8,NaN,NaN,2021-04-14 03:04:45,2021-04-14 04:35:10,2021-04-14 06:01:48,2021-04-14 07:30:33,2021-04-14 09:03:12
STHI_CGM_Potential_wrapper,NaN,NaN,"failed, exit code = 1","failed, exit code = 1","failed, exit code = 1","failed, exit code = 1","failed, exit code = 1"


In [11]:
#Highlight
def highlight_last_error(x):
    df = x.copy()
    df.loc[:,:] = 'background-color: white'
    df['Last log with error'] = 'background-color: yellow'
    for i in df.columns:
        df[i][x[i].astype(str).str.contains('.*failed.*') == True] = 'background-color: yellow'
    return df
Full_Report = Full_Report.style.apply(highlight_last_error, axis = None).set_properties(**{'border': '1px solid black'})

In [457]:
#Exporting
html = Full_Report.render()
with open("Full_Report.html","w") as fp:
    fp.write(html)